# Upstream Regulator Analysis Package Case Study

----------------------

Author: Mikayla Webster (m1webste@ucsd.edu)

Date: 8th November, 2017

----------------------

<a id='toc'></a>
## Table of Contents
1. [Import packages](#import)
2. [Create Graph](#graph)
3. [Background](#background)
    1. [Transcription Factors](#tf)
    2. [Background Network](#bn)
    3. [Differencially Expressed Genes](#deg)
3. [Stat Analysis](#stat)
    1. [P-value](#pvalue)
    2. [Z-score](#zscore)

## Import packages
<a id='import'></a>

In [8]:
import sys
code_path = 'upstream_regulator_analysis_project/ura'
sys.path.append(code_path)
import create_graph
import stat_analysis
reload(create_graph)
reload(stat_analysis)

<module 'stat_analysis' from 'upstream_regulator_analysis_project/ura\stat_analysis.py'>

## Background
<a id='background'></a>

The inspiration for these modules comes from [Ingenuity](http://pages.ingenuity.com/rs/ingenuity/images/0812%20upstream_regulator_analysis_whitepaper.pdf).

## Create Graph ##
<a id='graph'></a>

### Transcription Factors
<a id='tf'></a>

Our create_graph module prepares 3 databases to use in our analysis: a transcription factor interaction network, a background network, and a differencially expressed genes database. Our transcription factor data comes from two sources: [slowkow](https://github.com/slowkow/tftargets) and [jaspar](http://jaspar.genereg.net/). Slowkow is a compilation of 6 smaller databases: TRED, ITFP, ENCODE, Neph2012, TRRUST, Marbach2016. Information about these specific databases, including links to their sources, is available through the slowkow github page. Some basic information on these databases is as follows:
- Size of Jaspar Database: 2049
- Size of Slowkow Database: 2705
- Size of TRED: 133
- Size of ITFP: 1974
- Size of ENCODE: 157
- Size of Neph2012: 536
- Size of TRRUST: 748
- Size of Marbach2016: 643 

Our trascription factor list contains 3983 unique entries.

In [2]:
TF_list = create_graph.create_TF_list() # loads slowkow databases
len(TF_list)

3984

### Background Network
<a id='bn'></a>

Our background network comes from a subset of the [STRING](https://string-db.org/) database, and contains 40,216 interactions. We use our list of transcription factors (TF_list) to extract only relevant information from the STRING database. Namely, we pull out edge information from STRING only for our transcription factors (the entries in TF_list) and use those edges to create a graph. After this filtering process, our graph (labeled DG) has 687 nodes and 1505 edges.

In [3]:
STRING_DF, db_edges, db_sign_att = create_graph.load_and_process_STRING()
print 'Total number of interactions in STRING before filtering: ' + str(len(STRING_DF))
edge_list_filtered, sign_att_list_filtered = create_graph.filter_background(db_edges, db_sign_att, TF_list)
DG = create_graph.make_digraph(db_edges, db_sign_att, TF_list)
print 'Number of unique nodes after filtering: ' + str(len(DG.nodes()))
print 'Number of edges after filtering: ' + str(len(DG.edges()))

Total number of interactions in STRING before filtering: 40216
Number of unique nodes after filtering: 687
Number of edges after filtering: 1505


### Differencially Expressed Genes
<a id='deg'></a>

Our last database comes from a list of differencially exressed genes (DEG) found in an experiement. These are the genes we wish to analyze using our stat_analysis module. The TF graph created previously serves as our point of referece for these DEG's. Our DEG list (DEG_list) contains 2782 genes. 

In [4]:
DEG_list = create_graph.add_updown_from_DEG(DG)
len(DEG_list)

2782

## Stat Analysis

Our stat_analysis package can help identify TF's that are statistically significant to our set of DEG's.

### P-value

Our p-value function calculates the log of the p-value for every TF in the graph using [scipy.stats.hypergeom.logsf](https://docs.scipy.org/doc/scipy-0.19.1/reference/generated/scipy.stats.hypergeom.html). These values help us determine which TF's are actually associated with our DEG's. If a TF is given a high value (because we are working with logs, not straight p-values), then it is likely that there is correlation between that TF and its DEG targets. Therefore, it is likely that TF is responsible for some of our observed gene expression. Note that if a TF is given a value of zero, that means none of the TF's targets were DEG's. 

In [5]:
stat_analysis.tr_pvalues(DG, db_edges, DEG_list)

{u'ABD-B': 0,
 u'ACHI': 0,
 u'AKT1': 0,
 u'ANTP': 0,
 u'ARR1': 0,
 u'ATF6': 0,
 u'BAP': 0,
 u'BUB3': 0,
 u'CDC16': 3.144198753053967,
 u'CDC27': 2.0214533281294007,
 u'CDC6': 3.9095518809588667,
 u'CG11294': 0,
 u'CNOT4': 0,
 u'DFD': 0,
 u'DL': 5.3485051248380939,
 u'ECD': 0,
 u'EVE': 0,
 u'FKH': 0,
 u'GATA': 0,
 u'GCM': 0,
 u'GSTO1': 0,
 u'H': 0,
 u'HBN': 0,
 u'HDAC3': 0,
 u'HDAC6': 0,
 u'HKB': 0,
 u'ILK': 0,
 u'ING3': 0,
 u'INR': 0,
 u'INTS4': 5.6306387511574956,
 u'INTS6': 6.0258370197536495,
 u'INTS8': 6.5263844837146863,
 u'KLHL18': 0,
 u'KR': 0,
 u'LIG3': 0,
 u'MAD': 0,
 u'MARS': 0,
 u'MAX': 0,
 u'MCM2': 3.9716418433383445,
 u'MCM3': 5.3485051248380939,
 u'MCM5': 4.0535277071393567,
 u'MCM6': 6.8878747785318186,
 u'MCM7': 3.7766142159274736,
 u'MED1': 3.5382108565536048,
 u'MED15': 4.9811500142924485,
 u'MRPL23': 0,
 u'MRPL24': 11.39048554520299,
 u'MRPL44': 0,
 u'MSH6': 0,
 u'MYB': 0,
 u'NF1': 0,
 u'NUB': 0,
 u'NUP133': 0,
 u'NUP50': 0,
 u'ONECUT': 0,
 u'OPTIX': 0,
 u'PAN': 0,
 

### Z-score

The goal of our z-score function is to predict the activation states of the TF's. We observe how a TF relates to each of its targets to make our prediction. We compare each targets' observed gene regulation (either up or down) and each TF-target interation (whether it is activating or inhibiting) to conclude whether a TF is activating or inhibiting. A positive value indicates activating while a negative value indicates inhibiting. A value of zero means that we did not have enough information about the target or TF-target interaction to make the prediction. 

In [12]:
not_biased_zcsores = stat_analysis.tr_zscore(DG, DEG_list)
not_biased_zcsores

{u'ABD-B': 0,
 u'ACHI': 0,
 u'AKT1': 0,
 u'ANTP': 0,
 u'ARR1': 0,
 u'ATF6': 0,
 u'BAP': 0,
 u'BUB3': 0,
 u'CDC16': -1.0,
 u'CDC27': -1.0,
 u'CDC6': -1.0,
 u'CG11294': 0,
 u'CNOT4': 0,
 u'DFD': 0,
 u'DL': 0.0,
 u'ECD': 0,
 u'EVE': 0,
 u'FKH': 0,
 u'GATA': 0,
 u'GCM': 0,
 u'GSTO1': 0,
 u'H': 0,
 u'HBN': 0,
 u'HDAC3': 0,
 u'HDAC6': 0,
 u'HKB': 0,
 u'ILK': 0,
 u'ING3': 0,
 u'INR': 0,
 u'INTS4': -1.0,
 u'INTS6': -1.0,
 u'INTS8': -1.0,
 u'KLHL18': 0,
 u'KR': 0,
 u'LIG3': 0,
 u'MAD': 0,
 u'MARS': 0,
 u'MAX': 0,
 u'MCM2': -1.414213562373095,
 u'MCM3': -1.414213562373095,
 u'MCM5': -1.0,
 u'MCM6': -0.5773502691896258,
 u'MCM7': -1.0,
 u'MED1': -1.0,
 u'MED15': -1.414213562373095,
 u'MRPL23': 0,
 u'MRPL24': -1.7320508075688774,
 u'MRPL44': 0,
 u'MSH6': 0,
 u'MYB': 0,
 u'NF1': 0,
 u'NUB': 0,
 u'NUP133': 0,
 u'NUP50': 0,
 u'ONECUT': 0,
 u'OPTIX': 0,
 u'PAN': 0,
 u'PAX': 0.0,
 u'PNR': 0,
 u'PTEN': -1.0,
 u'PXN': 0,
 u'RAE1': 0,
 u'RBBP5': 0,
 u'REL': -1.0,
 u'REPO': 0,
 u'RFC4': -1.0,
 u'RNPS1': -1

In [13]:
TR_to_bias = stat_analysis.calculate_bias(DG, DEG_list)
TR_to_bias

{u'ABD-B': 0,
 u'ACHI': 0,
 u'AKT1': 0,
 u'ANTP': 0,
 u'ARR1': 0,
 u'ATF6': 0,
 u'BAP': 0,
 u'BUB3': 0,
 u'CDC16': -1.0,
 u'CDC27': -1.0,
 u'CDC6': -1.0,
 u'CG11294': 0,
 u'CNOT4': 0,
 u'DFD': 0,
 u'DL': 0.0,
 u'ECD': 0,
 u'EVE': 0,
 u'FKH': 0,
 u'GATA': 0,
 u'GCM': 0,
 u'GSTO1': 0,
 u'H': 0,
 u'HBN': 0,
 u'HDAC3': 0,
 u'HDAC6': 0,
 u'HKB': 0,
 u'ILK': 0,
 u'ING3': 0,
 u'INR': 0,
 u'INTS4': -1.0,
 u'INTS6': -1.0,
 u'INTS8': -1.0,
 u'KLHL18': 0,
 u'KR': 0,
 u'LIG3': 0,
 u'MAD': 0,
 u'MARS': 0,
 u'MAX': 0,
 u'MCM2': -1.0,
 u'MCM3': -1.0,
 u'MCM5': -1.0,
 u'MCM6': -0.3333333333333333,
 u'MCM7': -1.0,
 u'MED1': -1.0,
 u'MED15': -1.0,
 u'MRPL23': 0,
 u'MRPL24': -1.0,
 u'MRPL44': 0,
 u'MSH6': 0,
 u'MYB': 0,
 u'NF1': 0,
 u'NUB': 0,
 u'NUP133': 0,
 u'NUP50': 0,
 u'ONECUT': 0,
 u'OPTIX': 0,
 u'PAN': 0,
 u'PAX': -0.0,
 u'PNR': 0,
 u'PTEN': -1.0,
 u'PXN': 0,
 u'RAE1': 0,
 u'RBBP5': 0,
 u'REL': -1.0,
 u'REPO': 0,
 u'RFC4': -1.0,
 u'RNPS1': -1.0,
 u'RPL23A': 0.6,
 u'RPL8': 0.2222222222222222,
 u'RP

In [14]:
bias_zscores = stat_analysis.bias_corrected_tr_zscore(DG, DEG_list, TR_to_bias)
bias_zscores

{u'ABD-B': 0,
 u'ACHI': 0,
 u'AKT1': 0,
 u'ANTP': 0,
 u'ARR1': 0,
 u'ATF6': 0,
 u'BAP': 0,
 u'BUB3': 0,
 u'CDC16': 0.0,
 u'CDC27': 0.0,
 u'CDC6': 0.0,
 u'CG11294': 0,
 u'CNOT4': 0,
 u'DFD': 0,
 u'DL': -1.0,
 u'ECD': 0,
 u'EVE': 0,
 u'FKH': 0,
 u'GATA': 0,
 u'GCM': 0,
 u'GSTO1': 0,
 u'H': 0,
 u'HBN': 0,
 u'HDAC3': 0,
 u'HDAC6': 0,
 u'HKB': 0,
 u'ILK': 0,
 u'ING3': 0,
 u'INR': 0,
 u'INTS4': 0.0,
 u'INTS6': 0.0,
 u'INTS8': 0.0,
 u'KLHL18': 0,
 u'KR': 0,
 u'LIG3': 0,
 u'MAD': 0,
 u'MARS': 0,
 u'MAX': 0,
 u'MCM2': 0.0,
 u'MCM3': 0.0,
 u'MCM5': 0.0,
 u'MCM6': -2.2204460492503131e-16,
 u'MCM7': 0.0,
 u'MED1': 0.0,
 u'MED15': 0.0,
 u'MRPL23': 0,
 u'MRPL24': 0.0,
 u'MRPL44': 0,
 u'MSH6': 0,
 u'MYB': 0,
 u'NF1': 0,
 u'NUB': 0,
 u'NUP133': 0,
 u'NUP50': 0,
 u'ONECUT': 0,
 u'OPTIX': 0,
 u'PAN': 0,
 u'PAX': 0.0,
 u'PNR': 0,
 u'PTEN': 0.0,
 u'PXN': 0,
 u'RAE1': 0,
 u'RBBP5': 0,
 u'REL': 0.0,
 u'REPO': 0,
 u'RFC4': 0.0,
 u'RNPS1': 0.0,
 u'RPL23A': 2.7158479847794479,
 u'RPL8': 9.3822741353567842,
 u'

In [15]:
# comparison of unbiased z-score vs biased z-score calculation
# each entry is of form (bias, unbiased z-score, biased z-score)
zip(TR_to_bias.values(), not_biased_zcsores.values(), bias_zscores.values())

[(0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (-0.0, 0.0, 0.0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (-1.0, -1.0, 0.0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (-1.0, -1.7320508075688774, 0.0),
 (0.2222222222222222, 1.6329931618554523, 9.3822741353567842),
 (0, 0, 0),
 (-1.0, -1.0, 0.0),
 (-0.0, 0.0, 0.31783724519577983),
 (0, 0, 0),
 (-1.0, -1.0, 0.0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (-1.0, -1.0, 0.0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (1.0, 1.0, 0.0),
 (0.0, 0.0, 1.0352761804100798),
 (-1.0, -1.0, 0.0),
 (1.0, 1.0, 0.0),
 (0.0, 0.0, 2.0),
 (-1.0, -1.0, 0.0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (-1.0, -1.0, 0.0),
 (0, 0, 0),
 (0, 0, 0),
 (1.0, 1.0, 0.0),
 (0.0, 0.0, 0.31783724519577983),
 (0.30612244897959184, 1.889822365046136,